In [1]:
import os
import pandas as pd 
import math
import numpy as np 
import matplotlib.pyplot as plt
import random as r 
import glob
import seaborn as sns
from sklearn import metrics
from collections import defaultdict
from functools import partial
from itertools import repeat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
pd.options.mode.chained_assignment = None  # default='warn'

Using TensorFlow backend.


In [14]:
simfilepath = "/nBodyData/brutusSim/batch_brutus9_1.csv"
workDir = '/nBodyData/'

cols = ["eventID" ,"m1", "m2", "m3", 
         "x1", "x2", "x3", "y1", "y2", "y3", "dx1", "dx2", "dx3", "dy1", "dy2", "dy3",
         "tEnd",
         "x1f", "x2f", "x3f", "y1f", "y2f", "y3f", "dx1f", "dx2f", "dx3f", "dy1f", "dy2f", "dy3f"]

inputCols = ["m1", "m2", "m3", "x1", "x2", "x3", "y1", "y2", "y3", "dx1", "dx2", "dx3", "dy1", "dy2", "dy3"]

df = pd.read_csv(simfilepath,names=cols,index_col=False)
df.eventID = df.eventID.astype(int)

df_new = pd.DataFrame(columns = inputCols)

n_div = 0
n_good = 0
div_col = []

IndexError: list index out of range

In [12]:
print(df)

           eventID         m1         m2        m3        x1        x2  \
1 10001         84  56.314115  52.279696 -2.226488 -3.406637  9.440128   
  10002         84  56.314115  52.279696 -2.226488 -3.406637  9.440128   
  10003         84  56.314115  52.279696 -2.226488 -3.406637  9.440128   
  10004         84  56.314115  52.279696 -2.226488 -3.406637  9.440128   
  10005         84  56.314115  52.279696 -2.226488 -3.406637  9.440128   
...            ...        ...        ...       ...       ...       ...   
  6122556       84  69.451447   7.514575 -2.270110 -3.588650 -0.570330   
  6122557       84  69.451447   7.514575 -2.270110 -3.588650 -0.570330   
  6122558       84  69.451447   7.514575 -2.270110 -3.588650 -0.570330   
  6122559       84  69.451447   7.514575 -2.270110 -3.588650 -0.570330   
  6122560       84  69.451447   7.514575 -2.270110 -3.588650 -0.570330   

                 x3        y1        y2        y3  ...       x3f       y1f  \
1 10001   -3.887835 -4.902674 -2.

In [11]:
for i in range(int((df.shape[0]/2560))):
    
    index_i = int((i+1)*10000)
    index_f = int(((i+1)*10000)+2560)
    
    print(df[(df['eventID'] >= index_i) & (df['eventID'] <= (index_f))])
    event_pdf = df[(df['eventID'] >= index_i) & (df['eventID'] <= (index_f))]
    
#     print(event_pdf.shape[0])

    if(event_pdf.shape[0] < 1): continue

    tmp_df = event_pdf[inputCols]
    df_new.loc[i] = tmp_df.iloc[0,:]

    if(event_pdf.shape[0]<2560):
        div_col.append("1")
        n_div += 1
    else:
        div_col.append("0") 
        n_good += 1
        
print(len(div_col),df_new.shape)
df_new["div"] = div_col

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

...

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind

Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Index: []

[0 rows x 29 columns]
Empty DataFrame
Columns: [eventID, m1, m2, m3, x1, x2, x3, y1, y2, y3, dx1, dx2, dx3, dy1, dy2, dy3, tEnd, x1f, x2f, x3f, y1f, y2f, y3f, dx1f, dx2f, dx3f, dy1f, dy2f, dy3f]
Ind